In [3]:
from bs4 import BeautifulSoup
import pandas as pd
#import urllib2
import csv

In [9]:
def assignNoLabelStatus(s):
    status = "Label"
    s = s.strip(',').split(',')
    t = s[0]
    if len(t) > 1:
        # case to discard, eg C123G
        if t[0] in list('ACGT') and t[-1] in list('ACGT'):
            status = "NoLabel"
        # case to discard, eg C123G!
        elif t[0] in list('ACGT') and t[-1] == "!":
            status = "NoLabel"
        elif t[0].isalpha() == False:
            status = "NoLabel"
        # This last case replaces the next ones
        # case to discard, eg deletion 8123-8234d
        #elif t[0].isdigit() and t[-1] == "d":
        #    status = "NoLabel"
        # eg (C64T)
        #elif t[0] == "(":
        #    status = "NoLabel"
    return status

def checkIfAC(s):
    if len(s) > 0:
        if s[0].isalpha() and s[-1].isdigit():
            return True
        else:
            return False
    else:
        return False

In [4]:
#url = 'http://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3_en.php?block_no=47401&view=1'
#html = urllib2.urlopen(url).read()
html = open('mtDNA_tree_Build_17.onlytable.htm', 'r', encoding='utf-8').read()
#print(repr(html))

In [5]:
soup = BeautifulSoup(html, 'lxml')

In [6]:
table = soup.find_all("table")[0]

In [7]:
rows = table.find_all("tr")

In [8]:
# col_normal = rows[6]

In [10]:
csv_list = []
o = open('phylotree_17_190121.csv', 'w')
for row in rows:
    s = ""
    columns = row.find_all('td')
    for column in columns:
        column_text = ','.join(column.get_text().split())
        s += "{},".format(column_text)
        s = s.replace("\A0", "")
    if s.strip(',') != "":
        #print(s)
        #print(s.rstrip(','))
        s_list = s.rstrip(',').split(',')#[:-1]
        #print(s_list)
        # Check if it's a NoLabel
        for x,i in enumerate(s_list):
            if i != "":
                if i == 'L0a1a' or i == "L0":
                    print(s)
                status = assignNoLabelStatus(s)
                if status == "NoLabel":
                    #print("Found NoLabel at {}".format(','.join(s_list)))
                    s_list[x-1] = status
                    break
        # Delete ACs at the end
        #print(s_list)
        #try:
        while checkIfAC(s_list[-1]) == True:
            #print("Gonna reduce this: {}".format(s_list))
            s_list = s_list[:-1]
        while s_list[-1] == "":
            s_list = s_list[:-1]
        #except:
#            print("Error for:\n{}".format(s_list))
            #print(s_list)
        s = ','.join(s_list)
        if status == "NoLabel":
            print(s)
        csv_list.append(s)
        o.write(s+"\n")
o.close()

L0,G263A,C1048T,C3516a,T5442C,T6185C,C9042T,A9347G,G10589A,G12007A,A12720G,,,,,,,,,,,,,,,,,,,
,,,,,,,,L0a1a,(C64T),T3866C,,,,,,,,,,,,,,,EU092665,EU092764,
,,,,,,,,,NoLabel,A200G
,,,,,,,,NoLabel,A16293G
,,,,,,NoLabel,C152T
,,,,,NoLabel,A189G
,,,,,,,,NoLabel,T198C!
,,,,,,,,NoLabel,T16189C!,(C16192T)
,,,,,,,,,,NoLabel,G143A
,,,,,,,,NoLabel,G143A
,,,,,,,,,,NoLabel,T16086C
,,,,,,,,,,NoLabel,G16129A!
,,,,,,,,,NoLabel,G16309A!
,,,,,,,,,NoLabel,T16189C!,(C16192T)
,,,,,,,,,,NoLabel,G16309A!
,,,,,,,NoLabel,G16129A!
,,,,,,,NoLabel,G709A
,,,,,,,,,,NoLabel,T152C!
,,,,,,,,,,NoLabel,C16124T!
,,,,,,,,,,NoLabel,C16292T
,,,,,,,,,,,NoLabel,C150T
,,,,,,,,,,NoLabel,T152C!
,,,,,,,,,,NoLabel,T195C!
,,,,,,,,,NoLabel,T16311C!
,,,,,,,,,,,NoLabel,T16093C
,,,,,,,,,,,,NoLabel,G207A
,,,,,,,,,,,,,,NoLabel,T16093C
,,,,,,,,,,NoLabel,T204C
,,,,,,,,,NoLabel,T152C!
,,,,,,,,NoLabel,T16311C!
,,,,,,,,,,,NoLabel,C16311T!!
,,,,,,,,,,NoLabel,T195C!
,,,,,,,,,,NoLabel,T199C
,,,,,,,,,NoLabel,C16234T
,,,,,,,,,NoLabel,T152C!
,,,,,,